### Quick Run Block

In [1]:
# import torch

# # Define your checkpoint file path
# checkpoint_path = r"c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\pre_trained_models\reid\dukemtmcreid_resnet50_256_128_epoch_120.ckpt"
# new_checkpoint_path = checkpoint_path.replace(".ckpt", "_updated.ckpt")  # Save new version

# # Load the checkpoint with weights_only=False
# checkpoint = torch.load(checkpoint_path, weights_only=False)  # Ensures all data is loaded

# # Save the checkpoint again in an updated format
# torch.save(checkpoint, new_checkpoint_path)

# print(f"Checkpoint upgraded and saved to: {new_checkpoint_path}")

## Code

In [2]:
import sys
from pathlib import Path
import os

sys.path.append(str(Path.cwd().parent.parent))
print(str(Path.cwd().parent.parent))
print("Current working directory: ", os.getcwd())

from ModelDevelopment.CentralPipeline import CentralPipeline
from ModelDevelopment.ImageBatchPipeline import ImageBatchPipeline
from DataProcessing.DataPreProcessing import DataPaths

c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition
Current working directory:  c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\ModelDevelopment\experiments


## Multithreaded Studies Research Findings
- ThreadPoolExecutor works best for CentralPipeline pre-processing step (i.e. everything before run_pose)
- Time for one batch of 32 tracklets = 5 minutes, implying 9.3 seconds per tracklet, so ETA = (9.3*1210)/3600 = 3 hours pre-processing step execution
- (new, after image-level batching in reid centroid): 7s per tracklet => 2.35 hours w/ GPU gate=1
- (new, after above change + GPU gate=2): 6s. GPU memory usage: 2.0/8.0 GB. Suggested increase to gate=4
- GPU begins to crash after some time even with a gate bottleneck setting of 1 (only offload 1 tracklet to the GPU at a time)
- Solution to GPU crashing is to just let the process run once, then run it again with use_cache=True to catch up failed tracklets (will be fast as few fail)

In [3]:
# TODO: Add tracklets_override to pass a list of tracklets to process; tracklets_override=["8", "9", "10"]
# NOTE: It is recommended you delete the entire processed_data/{challenge/test/train} before running this to 100% avoid issues with old data.
# Furthermore, you should always restart your kernel before every new run because sometimes there are problems with paths
pipeline = CentralPipeline(
  num_tracklets=1210,
  #num_images_per_tracklet=1,
  input_data_path=DataPaths.TEST_DATA_DIR.value,
  output_processed_data_path=DataPaths.PROCESSED_DATA_OUTPUT_DIR_TEST.value,
  common_processed_data_dir=DataPaths.COMMON_PROCESSED_OUTPUT_DATA_TEST.value,
  gt_data_path=DataPaths.TEST_DATA_GT.value,
  single_image_pipeline=False,
  display_transformed_image_sample=False, # NOTE: DO NOT USE. Code is parallelized so we cannot show images anymore. Code breaks, but first one will show if True.
  num_image_samples=1,
  use_cache=True, # Set to false if you encounter data inconsistencies.
  suppress_logging=False,
  num_workers=6, # CRITICAL optimisation param. Anything more than 6 and your PC explodes.
  tracklet_batch_size=32 # CRITICAL optimisation param. 32 is a nice spot.
  )

2025-03-23 11:58:32 [INFO] DataPreProcessing initialized. Universe of available data paths:
2025-03-23 11:58:32 [INFO] ROOT_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted
2025-03-23 11:58:32 [INFO] TEST_DATA_GT: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\test_gt.json
2025-03-23 11:58:32 [INFO] TRAIN_DATA_GT: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\train_gt.json
2025-03-23 11:58:32 [INFO] TEST_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\images
2025-03-23 11:58:32 [INFO] TRAIN_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images
2025-03-23 11:58:32 [INFO] CHALLENGE_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\challenge\images
2

In [ ]:
pipeline.run_soccernet(
  run_soccer_ball_filter=True,
  generate_features=True,
  run_filter=True,
  run_legible=True,
  run_legible_eval=True,
  run_pose=False,
  run_crops=False,
  run_str=False,
  run_combine=False,
  run_eval=False)

2025-03-23 11:58:32 [INFO] Running the SoccerNet pipeline.


  0%|          | 0/38 [00:00<?, ?it/s]

2025-03-23 11:58:32 [INFO] Skipping preprocessing for tracklet: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\0
2025-03-23 11:58:32 [INFO] Skipping preprocessing for tracklet: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\1
2025-03-23 11:58:32 [INFO] Skipping preprocessing for tracklet: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\2
2025-03-23 11:58:32 [INFO] Skipping preprocessing for tracklet: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\3
2025-03-23 11:58:32 [INFO] Skipping preprocessing for tracklet: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\4
2025-03-23 11:58:32 [INFO] Skipping preprocessing for tracklet: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2

Processing tracklets (CUDA + CPU):   0%|          | 0/5 [00:00<?, ?it/s]

2025-03-23 11:58:43 [INFO] DEBUG: Number of images per tracklet in batch: 409
2025-03-23 11:58:43 [INFO] Creating placeholder data files for Soccer Ball Filter.
2025-03-23 11:58:43 [INFO] Creating placeholder data files for Legibility Classifier.
2025-03-23 11:58:43 [INFO] DEBUG: Just before getting args for workers
2025-03-23 11:58:43 [INFO] Determine soccer balls in image(s) using pre-trained model.
2025-03-23 11:58:43 [INFO] Determine soccer balls in image(s) using pre-trained model.
2025-03-23 11:58:43 [INFO] Determine soccer balls in image(s) using pre-trained model.
2025-03-23 11:58:43 [INFO] Determine soccer balls in image(s) using pre-trained model.
2025-03-23 11:58:43 [INFO] Determine soccer balls in image(s) using pre-trained model.


...

Lightning automatically upgraded your loaded checkpoint from v1.1.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\pre_trained_models\reid\dukemtmcreid_resnet50_256_128_epoch_120.ckpt`


2025-03-23 11:59:54 [INFO] Saved features for tracklet with shape (382, 2048) to c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\97\features.npy
2025-03-23 11:59:54 [INFO] Identifying and removing outliers by calling gaussian_outliers_streamlined.py on feature file


Lightning automatically upgraded your loaded checkpoint from v1.1.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\pre_trained_models\reid\dukemtmcreid_resnet50_256_128_epoch_120.ckpt`


2025-03-23 11:59:55 [INFO] Saved features for tracklet with shape (89, 2048) to c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\106\features.npy
2025-03-23 11:59:55 [INFO] Identifying and removing outliers by calling gaussian_outliers_streamlined.py on feature file


Lightning automatically upgraded your loaded checkpoint from v1.1.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\pre_trained_models\reid\dukemtmcreid_resnet50_256_128_epoch_120.ckpt`
